In [432]:
import pandas as pd
import numpy as np
import re
from typing import List
from gensim.models.phrases import Phrases, Phraser
from w2v_utils import (Tokenizer)
from gensim.models import Word2Vec
import multiprocessing

import json

import nltk 
#nltk.download('stopwords')

In [433]:
db = pd.read_csv(r'C:\Users\Daniel\Documents\IS IV\metody_i_narzedzia_big_data\L_K_Brzostowski\Projekt\fine\data\data.csv')
print(db.shape)
db.head()

(3101, 1)


,text
0,#BITCOIN DOESN'T GIVE A SHIT ABOUT TA! IT'S BE...
1,BITCOIN CRASH TO 30K?? THIS IS WHAT CHARTS TEL...
2,👉 Current average #Bitcoin transaction fee is ...
3,#btc pumping ... must be some twitter beef goi...
4,#fun #tumblr #twitter #facebook #instagram #do...


In [434]:
def remove_contractions(text):
  text = text.replace('’', "'")
  contractions_path = r'C:\Users\Daniel\Documents\IS IV\metody_i_narzedzia_big_data\L_K_Brzostowski\Projekt\fine\data\contractions.json'
  with open(contractions_path) as json_file:
      contractions = json.load(json_file)

  for word in text.split():
      if word.lower() in contractions:
          text = text.replace(word, contractions[word.lower()])
  return text

def denoise(text):
  """
  Removing html and URL components
  """
  html_pattern = r"<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});"
  url_pattern = r"((http://)[^ ]*|(https://)[^ ]*|(www\.)[^ ]*)"

  temp = text
  temp = re.sub("$[A-Za-z0-9_]+","", temp) # ?
  temp = re.sub("@[A-Za-z0-9_]+","", temp)  
  temp = re.sub("#[A-Za-z0-9_]+","", temp)
  temp = re.sub("&[A-Za-z0-9_]+","", temp)
  text = temp
  text = re.sub(html_pattern, " ", text)
  text = re.sub(url_pattern," ",text).strip()
  text = re.sub(r'[0-9]', '', text) 
  return text

def remove_stopwords(tokenized_text):
  stopwords = nltk.corpus.stopwords.words('english')
  text = [word for word in tokenized_text if word not in stopwords]
  return text

def tokenize(text, removeContractions=True, deNoise=True, lower=True, clean=True, removeStopWords=True):
        """
        A function to tokenize words of the text
        """
        non_alphanumeric_pattern =r"[^a-zA-Z0-9]"
        text = ' '.join(text.split())
        # to substitute multiple whitespace with single whitespace
        # text = ' '.join(text.split())
        if removeContractions:
            text = remove_contractions(text)
        if deNoise:
            text = denoise(text)
        if lower:
            text = text.lower()
            
        if clean:
            # to remove non-alphanumeric characters
            text = re.sub(non_alphanumeric_pattern," ", text).strip()

        tokenized_text = text.split()

        if removeStopWords:
            tokenized_text = remove_stopwords(tokenized_text)

        return tokenized_text


In [435]:
db['tokenized'] = db['text'].apply(lambda x: tokenize(x, removeStopWords=False))
db.shape



(3101, 2)

In [436]:
for i in range(db.shape[0]):
  print('------------------------')
  print()
  print(db['text'][i])
  print(db['tokenized'][i])

------------------------

#BITCOIN DOESN'T GIVE A SHIT ABOUT TA! IT'S BELOW ALL EMAs, MAs, ALL SUPPORTS, AND THE BREAKDOWN FROM BEAR FLAG!
AND IT STILL WILL PUMP BACK TO ATH!
['does', 'not', 'give', 'a', 'shit', 'about', 'ta', 'it', 'has', 'it', 'is', 'below', 'all', 'emas', 'mas', 'all', 'supports', 'and', 'the', 'breakdown', 'from', 'bear', 'flag', 'and', 'it', 'still', 'will', 'pump', 'back', 'to', 'ath']
------------------------

BITCOIN CRASH TO 30K?? THIS IS WHAT CHARTS TELL US...... https://t.co/XCwKkDEB2S via @YouTube #bitcoin https://t.co/j1Y2fDo9Sy
['bitcoin', 'crash', 'to', 'k', 'this', 'is', 'what', 'charts', 'tell', 'us', 'via']
------------------------

👉 Current average #Bitcoin transaction fee is $0.15 (1 input, 2 outputs, SegWit, 1 hour conf. time.)

You can save fees and time by using the #LightningNetwork⚡️! Start now!
['current', 'average', 'transaction', 'fee', 'is', 'input', 'outputs', 'segwit', 'hour', 'conf', 'time', 'you', 'can', 'save', 'fees', 'and', 'time', 

In [437]:
def drop_row(text):
  if len(text) < 3:
    return 0
  else: return 1


db['size'] = db['tokenized'].apply(drop_row)
db = db.drop(db[db['size'] < 1].index, inplace=False)
db = db.drop(['size'], axis=1)
print(db.shape)

(3011, 2)


In [438]:
# phrases = Phrases(db['tokenized'], min_count=3, progress_per=10000)
# bigram = Phraser(phrases)
# db['bigram'] = bigram[db['tokenized']]
# sentences = db['bigram']
sentences = db['tokenized']
sentences.shape



(3011,)

In [439]:
w2v_model = Word2Vec(min_count=5,
                     window=4,
                     vector_size=300,
                     sample=1e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=multiprocessing.cpu_count()-1)

w2v_model.build_vocab(sentences, progress_per=50000)


In [440]:
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

w2v_model.init_sims(replace=True)

C:\Users\Daniel\AppData\Local\Temp/ipykernel_21508/3364510996.py:3: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  w2v_model.init_sims(replace=True)


In [441]:
w2v_model.save("word2vec.model")

In [442]:
w2v_model.wv.most_similar(positive=["buy"])

[('scan', 0.9997082352638245),
 ('on', 0.9996758103370667),
 ('all', 0.9996738433837891),
 ('paxg', 0.9996685981750488),
 ('trx', 0.999668538570404),
 ('bnb', 0.9996684789657593),
 ('not', 0.9996528625488281),
 ('lunas', 0.9996510744094849),
 ('ltc', 0.9996510744094849),
 ('xtz', 0.9996508359909058)]

In [443]:
# db['bigram'] = db['bigram'].apply(lambda x: ' '.join(x))
# db['bigram'].to_csv('cleaned_data.csv', index=False)

db['tokenized'] = db['tokenized'].apply(lambda x: ' '.join(x))
db['tokenized'].to_csv('data\cleaned_data.csv', index=False)
